<a href="https://colab.research.google.com/github/Sourav56/Computer-Vision/blob/master/ssd_mask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
import os
import zipfile
import pandas as pd
import random
import shutil
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
from os import getcwd
from os import listdir
import cv2
from tensorflow.keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import imutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image  as mpimg

In [ ]:
directory=r'/content/drive/My Drive/Colab Notebooks/facemaskdetection'
category =  os.listdir(directory)
print(category)

['without_mask', 'with_mask']


In [ ]:
yes_im = r'/content/drive/My Drive/Colab Notebooks/facemaskdetection/with_mask'
no_im = r'/content/drive/My Drive/Colab Notebooks/facemaskdetection/without_mask'

In [ ]:
print("The number of images with facemask labelled 'yes':",len(os.listdir(yes_im)))
print("The number of images with facemask labelled 'no':",len(os.listdir(no_im)))

The number of images with facemask labelled 'yes': 1915
The number of images with facemask labelled 'no': 1918


In [ ]:

!git clone https://github.com/prajnasb/observations

Cloning into 'observations'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 1638 (delta 9), reused 0 (delta 0), pack-reused 1604
Receiving objects: 100% (1638/1638), 75.94 MiB | 9.35 MiB/s, done.
Resolving deltas: 100% (20/20), done.


In [ ]:
!ls observations/experiements/dest_folder/

test  test.csv	train  train.csv  val


In [ ]:
def data_summary(main_path):
    
    yes_path = main_path+'with_mask'
    no_path = main_path+'without_mask'
        
    # number of files (images) that are in the the folder named 'yes' that represent tumorous (positive) examples
    m_pos = len(listdir(yes_path))
    # number of files (images) that are in the the folder named 'no' that represent non-tumorous (negative) examples
    m_neg = len(listdir(no_path))
    # number of all examples
    m = (m_pos+m_neg)
    
    pos_prec = (m_pos* 100.0)/ m
    neg_prec = (m_neg* 100.0)/ m
    
    print(f"Number of examples: {m}")
    print(f"Percentage of positive examples: {pos_prec}%, number of pos examples: {m_pos}") 
    print(f"Percentage of negative examples: {neg_prec}%, number of neg examples: {m_neg}") 
    
augmented_data_path =  r'/content/drive/My Drive/Colab Notebooks/facemaskdetection/' 
data_summary(augmented_data_path)

Number of examples: 3833
Percentage of positive examples: 49.96086616227498%, number of pos examples: 1915
Percentage of negative examples: 50.03913383772502%, number of neg examples: 1918


In [ ]:
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
    dataset = []
    
    for unitData in os.listdir(SOURCE):
        data = SOURCE + unitData
        if(os.path.getsize(data) > 0):
            dataset.append(unitData)
        else:
            print('Skipped ' + unitData)
            print('Invalid file i.e zero size')
    
    train_set_length = int(len(dataset) * SPLIT_SIZE)
    test_set_length = int(len(dataset) - train_set_length)
    shuffled_set = random.sample(dataset, len(dataset))
    train_set = dataset[0:train_set_length]
    test_set = dataset[-test_set_length:]
       
    for unitData in train_set:
        temp_train_set = SOURCE + unitData
        final_train_set = TRAINING + unitData
        copyfile(temp_train_set, final_train_set)
    
    for unitData in test_set:
        temp_test_set = SOURCE + unitData
        final_test_set = TESTING + unitData
        copyfile(temp_test_set, final_test_set)
        
YES_SOURCE_DIR = "observations/experiements/data/with_mask/"
TRAINING_YES_DIR = "observations/experiements/dest_folder/train/with_mask/"
TESTING_YES_DIR = "observations/experiements/dest_folder/test/with_mask/"

NO_SOURCE_DIR =  "observations/experiements/data/without_mask/"
TRAINING_NO_DIR = "observations/experiements/dest_folder/train/without_mask/"
TESTING_NO_DIR = 'observations/experiements/dest_folder/test/without_mask/'
split_size = 0.8
split_data( YES_SOURCE_DIR, TRAINING_YES_DIR, TESTING_YES_DIR, split_size)
split_data( NO_SOURCE_DIR,TRAINING_NO_DIR, TESTING_NO_DIR, split_size)

In [ ]:
print("The number of images with facemask in the training set labelled 'yes':", len(os.listdir("observations/experiements/dest_folder/train/with_mask/")))
print("The number of images with facemask in the test set labelled 'yes':", len(os.listdir("observations/experiements/dest_folder/test/with_mask/")))
print("The number of images without facemask in the training set labelled 'no':", len(os.listdir("observations/experiements/dest_folder/train/without_mask/")))
print("The number of images without facemask in the test set labelled 'no':", len(os.listdir("observations/experiements/dest_folder/test/without_mask/")))

The number of images with facemask in the training set labelled 'yes': 729
The number of images with facemask in the test set labelled 'yes': 209
The number of images without facemask in the training set labelled 'no': 1091
The number of images without facemask in the test set labelled 'no': 213


In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(100, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(100, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [ ]:
TRAINING_DIR = "/content/drive/My Drive/Colab Notebooks/facemaskdetection/"
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(TRAINING_DIR, 
                                                    batch_size=10, 
                                                    target_size=(150, 150))
VALIDATION_DIR = "observations/experiements/dest_folder/test/"
validation_datagen = ImageDataGenerator(rescale=1.0/255)

validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR, 
                                                         batch_size=10, 
                                                         target_size=(150, 150))
checkpoint = ModelCheckpoint('model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')


Found 3833 images belonging to 2 classes.
Found 422 images belonging to 2 classes.


In [ ]:
history = model.fit_generator(train_generator,
                              epochs=10,
                              validation_data=validation_generator,
                              callbacks=[checkpoint])

Epoch 1/10
 22/384 [>.............................] - ETA: 27:18 - loss: 0.8829 - acc: 0.5045

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


384/384 [==============================] - 1635s 4s/step - loss: 0.4650 - acc: 0.7949 - val_loss: 0.4267 - val_acc: 0.8673
Epoch 2/10
384/384 [==============================] - 419s 1s/step - loss: 0.4013 - acc: 0.8482 - val_loss: 0.3600 - val_acc: 0.8649
Epoch 3/10
384/384 [==============================] - 420s 1s/step - loss: 0.3441 - acc: 0.8654 - val_loss: 0.2851 - val_acc: 0.8910
Epoch 4/10
384/384 [==============================] - 412s 1s/step - loss: 0.3321 - acc: 0.8669 - val_loss: 0.3042 - val_acc: 0.8744
Epoch 5/10
384/384 [==============================] - 419s 1s/step - loss: 0.3170 - acc: 0.8748 - val_loss: 0.2575 - val_acc: 0.8981
Epoch 6/10
384/384 [==============================] - 414s 1s/step - loss: 0.2976 - acc: 0.8865 - val_loss: 0.2450 - val_acc: 0.9100
Epoch 7/10
384/384 [==============================] - 411s 1s/step - loss: 0.3094 - acc: 0.8737 - val_loss: 0.3821 - val_acc: 0.8578
Epoch 8/10
384/384 [==============================] - 411s 1s/step - loss: 0.30

In [ ]:
face_clsfr=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [ ]:
with open('/trainHistoryDict', 'wb') as file_pi:
        pickle.dump(history.history, file_pi)

In [ ]:
def VideoCapture():
  js = Javascript('''
    async function create(){
      div = document.createElement('div');
      document.body.appendChild(div);

      video = document.createElement('video');
      video.setAttribute('playsinline', '');

      div.appendChild(video);

      stream = await navigator.mediaDevices.getUserMedia({video: {facingMode: "environment"}});
      video.srcObject = stream;

      await video.play();

      canvas =  document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);

      div_out = document.createElement('div');
      document.body.appendChild(div_out);
      img = document.createElement('img');
      div_out.appendChild(img);
    }

    async function capture(){
        return await new Promise(function(resolve, reject){
            pendingResolve = resolve;
            canvas.getContext('2d').drawImage(video, 0, 0);
            result = canvas.toDataURL('image/jpeg', 0.8);
            pendingResolve(result);
        })
    }

    function showimg(imgb64){
        img.src = "data:image/jpg;base64," + imgb64;
    }

  ''')
  display(js)

def byte2image(byte):
  jpeg = b64decode(byte.split(',')[1])
  im = Image.open(io.BytesIO(jpeg))
  return np.array(im)

def image2byte(image):
  image = Image.fromarray(image)
  buffer = io.BytesIO()
  image.save(buffer, 'jpeg')
  buffer.seek(0)
  x = b64encode(buffer.read()).decode('utf-8')
  return x


In [ ]:
!wget https://github.com/AKSHAYUBHAT/TensorFace/raw/master/openface/models/dlib/shape_predictor_68_face_landmarks.dat
!wget https://github.com/ageitgey/face_recognition_models/raw/master/face_recognition_models/models/dlib_face_recognition_resnet_model_v1.dat

--2020-08-26 20:44:40--  https://github.com/AKSHAYUBHAT/TensorFace/raw/master/openface/models/dlib/shape_predictor_68_face_landmarks.dat
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/AKSHAYUBHAT/TensorFace/master/openface/models/dlib/shape_predictor_68_face_landmarks.dat [following]
--2020-08-26 20:44:40--  https://raw.githubusercontent.com/AKSHAYUBHAT/TensorFace/master/openface/models/dlib/shape_predictor_68_face_landmarks.dat
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 99693937 (95M) [application/octet-stream]
Saving to: ‘shape_predictor_68_face_landmarks.dat’

shape_predictor_68_ 100%[======

In [ ]:
!wget https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml

--2020-08-26 20:45:32--  https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 930127 (908K) [text/plain]
Saving to: ‘haarcascade_frontalface_default.xml’

haarcascade_frontal 100%[===================>] 908.33K  --.-KB/s    in 0.07s   

2020-08-26 20:45:32 (12.4 MB/s) - ‘haarcascade_frontalface_default.xml’ saved [930127/930127]



In [ ]:
import numpy as np, cv2, dlib, os, pickle
path = './facedata/'
detector = dlib.get_frontal_face_detector()
sp = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
model = dlib.face_recognition_model_v1('dlib_face_recognition_resnet_model_v1.dat')
FACE_DESC = []
FACE_NAME = []
for fn in os.listdir(path):
    if fn.endswith('.jpg'):
        img = cv2.imread(path + fn)[:,:,::-1]
        dets = detector(img, 1)
        for k, d in enumerate(dets):
            shape = sp(img, d)
            face_desc = model.compute_face_descriptor(img, shape, 100)
            FACE_DESC.append(np.array(face_desc))
            print('loading...', fn)
            FACE_NAME.append(fn[:fn.index('_')])
pickle.dump((FACE_DESC, FACE_NAME), open('trainset.pk', 'wb'))

FileNotFoundError: ignored

In [ ]:
face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
detector = './facedata/'
sp = dlib.shape_predictor('')
model = dlib.face_recognition_model_v1('')
FACE_DESC, FACE_NAME = pickle.load(open('', 'rb'))

VideoCapture()
eval_js('create()')
while True:
    byte = eval_js('capture()')
    frame = byte2image(byte)
    gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    faces = face_detector.detectMultiScale(gray, 1.3, 5)
    for (x, y, w, h) in faces:
        img = frame[y-10:y+h+10, x-10:x+w+10][:,:,::-1]
        dets = detector(img, 1)
        for k, d in enumerate(dets):
            shape = sp(img, d)
            face_desc0 = model.compute_face_descriptor(img, shape, 1)
            d = []
            for face_desc in FACE_DESC:
                d.append(np.linalg.norm(np.array(face_desc) - np.array(face_desc0)))
            d = np.array(d)
            idx = np.argmin(d)
            if d[idx] < 0.5:
                name = FACE_NAME[idx]
                #print(name)
                cv2.putText(frame, name, (x, y-5), cv2.FONT_HERSHEY_COMPLEX, .7, (255,255,255),2)
                cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
    eval_js('showimg("{}")'.format(image2byte(frame)))

NameError: ignored